In [7]:
###############################
# TRAIN CODE
###############################

import os
import glob
import json
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from scipy.signal import savgol_filter
from geopy.distance import geodesic
import warnings
from scipy.stats import skew, kurtosis
from skopt import gp_minimize
from skopt.space import Real, Integer
from sklearn.ensemble import IsolationForest
from sklearn.utils import class_weight

warnings.filterwarnings("ignore", category=RuntimeWarning)

# -----------------------------
# 1. Load and Combine CSV Files (with potholes column)
# -----------------------------
folder_path = r"C:\Users\Rick Halder\Desktop\SpeedyCare\Best Work\Pothole_Non_Pothole"  # <-- Update this path
file_pattern = os.path.join(folder_path, "*.csv")
csv_files = glob.glob(file_pattern)

df_list = []
for file in csv_files:
    try:
        df = pd.read_csv(file)
        df["source_file"] = os.path.basename(file)
        df_list.append(df)
    except Exception as e:
        print(f"Error reading {file}: {e}")

if df_list:
    combined_data = pd.concat(df_list, ignore_index=True)
    print(f"Combined DataFrame shape: {combined_data.shape}")
else:
    raise ValueError("No CSV files were successfully read.")

# -----------------------------
# 2. Helper Functions
# -----------------------------
def apply_kalman_filter(series, process_variance=1e-5, measurement_variance=1e-2):
    n = len(series)
    estimates = np.zeros(n)
    error_estimate = 1.0
    error_measure = measurement_variance
    for i in range(n):
        error_estimate += process_variance
        kalman_gain = error_estimate / (error_estimate + error_measure)
        estimate = series[i] if i == 0 else estimates[i-1]
        estimate = estimate + kalman_gain * (series[i] - estimate)
        error_estimate = (1 - kalman_gain) * error_estimate
        estimates[i] = estimate
    return estimates

def z_thresh_detection(series, threshold=3):
    mean_val = series.mean()
    std_val = series.std()
    z_scores = (series - mean_val) / std_val
    anomaly_mask = (np.abs(z_scores) > threshold).astype(int)
    return anomaly_mask, z_scores

def isolation_forest_anomaly_detection(df, contamination=0.01):
    features = df[['acc_magnitude', 'jerk']]
    clf = IsolationForest(contamination=contamination, random_state=42)
    preds = clf.fit_predict(features)
    df['isof_anomaly'] = (preds == -1).astype(int)
    return df

def load_and_preprocess_df(df, process_variance=1e-5, measurement_variance=1e-2, 
                           sg_window_length=21, sg_polyorder=3):
    df = df.dropna(subset=['latitude', 'longitude'])
    df = df[np.isfinite(df['latitude']) & np.isfinite(df['longitude'])]
    if 'timestamp' in df.columns:
        try:
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
        except Exception:
            df['timestamp'] = pd.to_datetime(df['timestamp'])
    else:
        df['timestamp'] = pd.date_range(start='2025-01-01', periods=len(df), freq='s')
    
    # Rename columns if needed
    rename_dict = {}
    if 'accelerometerX' in df.columns:
        rename_dict['accelerometerX'] = 'acc_x'
    if 'accelerometerY' in df.columns:
        rename_dict['accelerometerY'] = 'acc_y'
    if 'accelerometerZ' in df.columns:
        rename_dict['accelerometerZ'] = 'acc_z'
    if 'gyroX' in df.columns:
        rename_dict['gyroX'] = 'gyro_x'
    if 'gyroY' in df.columns:
        rename_dict['gyroY'] = 'gyro_y'
    if 'gyroZ' in df.columns:
        rename_dict['gyroZ'] = 'gyro_z'
    if 'potholes' in df.columns:
        rename_dict['potholes'] = 'pothole_label'
    df = df.rename(columns=rename_dict)
    
    # Ensure required columns exist
    for col in ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z', 'pothole_label']:
        if col not in df.columns:
            df[col] = 0

    df['acc_x_raw'] = df['acc_x'].copy()
    sensor_cols = ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']
    for col in sensor_cols:
        filtered = apply_kalman_filter(df[col].values, process_variance, measurement_variance)
        if len(filtered) >= sg_window_length:
            df[col] = savgol_filter(filtered, window_length=sg_window_length, polyorder=sg_polyorder, mode='nearest')
        else:
            df[col] = filtered

    df['acc_magnitude'] = np.linalg.norm(df[['acc_x', 'acc_y', 'acc_z']], axis=1)
    df['gyro_magnitude'] = np.linalg.norm(df[['gyro_x', 'gyro_y', 'gyro_z']], axis=1)
    time_diff = df['timestamp'].diff().dt.total_seconds().fillna(1)
    df['jerk'] = np.gradient(df['acc_magnitude'], time_diff)
    
    distances = [0.0]
    for i in range(1, len(df)):
        prev_point = (df['latitude'].iloc[i-1], df['longitude'].iloc[i-1])
        curr_point = (df['latitude'].iloc[i], df['longitude'].iloc[i])
        distances.append(geodesic(prev_point, curr_point).meters)
    df['distance'] = distances
    
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df = df.ffill().bfill().dropna(subset=['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z', 'latitude', 'longitude'])
    
    scaler = RobustScaler()
    cols_to_scale = sensor_cols + ['acc_magnitude', 'gyro_magnitude', 'jerk']
    df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])
    
    df['z_anomaly'], df['z_score'] = z_thresh_detection(df['acc_magnitude'], threshold=3)
    df['z_diff'] = df['z_score'].diff().fillna(0)
    df['z_diff_anomaly'] = (np.abs(df['z_diff']) > 2).astype(int)
    window_size = 50
    df['rolling_mean'] = df['acc_magnitude'].rolling(window=window_size, min_periods=1).mean()
    df['rolling_std'] = df['acc_magnitude'].rolling(window=window_size, min_periods=1).std().fillna(0)
    df['std_anomaly'] = (np.abs(df['acc_magnitude'] - df['rolling_mean']) > 3 * (df['rolling_std'] + 1e-8)).astype(int)
    threshold_low = 0.1
    df['g_zero_flag'] = (df['acc_magnitude'] < threshold_low).astype(int)
    
    df = isolation_forest_anomaly_detection(df, contamination=0.01)
    return df

def extract_sliding_window_features(df, window_size=50, step=25):
    feature_list = []
    for start in range(0, len(df) - window_size + 1, step):
        window = df.iloc[start:start+window_size]
        center_idx = window_size // 2
        features = {
            'timestamp': window['timestamp'].iloc[center_idx],
            'latitude': window['latitude'].iloc[center_idx],
            'longitude': window['longitude'].iloc[center_idx],
            'acc_magnitude': window['acc_magnitude'].mean(),
            'acc_std': window['acc_magnitude'].std(),
            'acc_min': window['acc_magnitude'].min(),
            'acc_max': window['acc_magnitude'].max(),
            'acc_skew': skew(window['acc_magnitude']),
            'acc_kurtosis': kurtosis(window['acc_magnitude']),
            'jerk_mean': window['jerk'].mean(),
            'jerk_std': window['jerk'].std(),
            'z_anomaly_rate': window['z_anomaly'].mean(),
            'z_diff_rate': window['z_diff_anomaly'].mean(),
            'std_anomaly_rate': window['std_anomaly'].mean(),
            'g_zero_rate': window['g_zero_flag'].mean(),
            'isof_anomaly_rate': window['isof_anomaly'].mean(),
            'pothole_label': window['pothole_label'].iloc[center_idx]
        }
        feature_list.append(features)
    return pd.DataFrame(feature_list)

def create_sequences(features_df, feature_columns, sequence_length=10):
    features_df = features_df.sort_values(by='timestamp').reset_index(drop=True)
    X_seq = []
    for i in range(len(features_df) - sequence_length + 1):
        seq = features_df.iloc[i:i+sequence_length]
        X_seq.append(seq[feature_columns].values)
    return np.array(X_seq), features_df

# -----------------------------
# 3. Combined Tuning of Filtering Parameters (Dynamic)
# -----------------------------
# Use raw accelerometer data for tuning; prefer the 'acc_x_raw' column if available.
if 'acc_x_raw' in combined_data.columns:
    sensor_series_raw = combined_data['acc_x_raw'].values
elif 'acc_x' in combined_data.columns:
    sensor_series_raw = combined_data['acc_x'].values
else:
    np.random.seed(42)
    t_sim = np.linspace(0, 10, 500)
    sensor_series_raw = np.sin(t_sim) + np.random.normal(0, 0.2, t_sim.shape)

def tuning_cost(params):
    process_variance, measurement_variance, sg_window_length, sg_polyorder = params
    sg_window_length = int(sg_window_length)
    if sg_window_length % 2 == 0:
        sg_window_length += 1
    sg_polyorder = int(sg_polyorder)
    if sg_polyorder >= sg_window_length:
        return 1e6
    kalman_filtered = apply_kalman_filter(sensor_series_raw, process_variance, measurement_variance)
    sg_filtered = savgol_filter(kalman_filtered, window_length=sg_window_length, polyorder=sg_polyorder, mode='nearest')
    residual = sensor_series_raw - sg_filtered
    mae = np.mean(np.abs(residual))
    nis = np.mean(((sensor_series_raw - sg_filtered) ** 2) / measurement_variance)
    nis_error = np.abs(nis - 1)
    return mae + nis_error

space = [
    Real(1e-6, 1e-3, prior='log-uniform', name='process_variance'),
    Real(1e-3, 1e-1, prior='log-uniform', name='measurement_variance'),
    Integer(7, 51, name='sg_window_length'),
    Integer(1, 5, name='sg_polyorder')
]

result = gp_minimize(tuning_cost, space, n_calls=50, random_state=42)
opt_process_variance    = result.x[0]
opt_measurement_variance  = result.x[1]
opt_sg_window_length    = int(result.x[2])
if opt_sg_window_length % 2 == 0:
    opt_sg_window_length += 1
opt_sg_polyorder        = int(result.x[3])

print("Optimized Filtering Parameters:")
print(" Process Variance:", opt_process_variance)
print(" Measurement Variance:", opt_measurement_variance)
print(" SG Window Length:", opt_sg_window_length)
print(" SG Polyorder:", opt_sg_polyorder)

# Save filtering parameters for use during inference.
filtering_params = {
    "process_variance": opt_process_variance,
    "measurement_variance": opt_measurement_variance,
    "sg_window_length": opt_sg_window_length,
    "sg_polyorder": opt_sg_polyorder
}
with open("filtering_params.json", "w") as f:
    json.dump(filtering_params, f)
print("Filtering parameters saved to filtering_params.json")

# -----------------------------
# 4. Preprocess Data and Extract Features
# -----------------------------
combined_data_processed = load_and_preprocess_df(
    combined_data,
    process_variance=opt_process_variance,
    measurement_variance=opt_measurement_variance,
    sg_window_length=opt_sg_window_length,
    sg_polyorder=opt_sg_polyorder
)
print("Processed data shape:", combined_data_processed.shape)

features_df = extract_sliding_window_features(combined_data_processed, window_size=50, step=25)
print("Extracted features shape:", features_df.shape)

feature_columns = [
    'acc_magnitude', 'acc_std', 'acc_min', 'acc_max', 
    'acc_skew', 'acc_kurtosis',
    'jerk_mean', 'jerk_std',
    'z_anomaly_rate', 'z_diff_rate', 'std_anomaly_rate', 
    'g_zero_rate', 'isof_anomaly_rate'
]
sequence_length = 10
X_seq, features_df = create_sequences(features_df, feature_columns, sequence_length=sequence_length)
print("Sequence shape:", X_seq.shape, "Labels shape:", features_df.shape)

# -----------------------------
# 5. Create Labels and Train-Test Split
# -----------------------------
# Label each sequence as 1 if any window in the sequence shows a pothole.
y_seq = []
for i in range(len(features_df) - sequence_length + 1):
    seq = features_df.iloc[i:i+sequence_length]
    y_seq.append(1 if seq['pothole_label'].max() >= 1 else 0)
y_seq = np.array(y_seq)

X_train_seq, X_val_seq, y_train_seq, y_val_seq = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)
class_weights_seq = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_seq), y=y_train_seq)
class_weights_seq = dict(enumerate(class_weights_seq))

# -----------------------------
# 6. Build and Train the Hybrid CNN-RNN Model
# -----------------------------
def build_hybrid_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.LSTM(64)(x)
    x = layers.Dense(32, activation='relu')(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.Recall(name='recall'),
                           tf.keras.metrics.Precision(name='precision')])
    return model

hybrid_model = build_hybrid_model(X_train_seq.shape[1:])
print("Training Hybrid CNN-RNN Model...")
history = hybrid_model.fit(
    X_train_seq, y_train_seq,
    epochs=50,
    batch_size=32,
    validation_data=(X_val_seq, y_val_seq),
    callbacks=[callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)],
    class_weight=class_weights_seq,
    verbose=1
)

loss, acc, rec, prec = hybrid_model.evaluate(X_val_seq, y_val_seq, verbose=0)
print("Validation Results:")
print(f"Loss: {loss:.4f}, Accuracy: {acc:.4f}, Recall: {rec:.4f}, Precision: {prec:.4f}")

# -----------------------------
# 7. Save the Trained Model in Native Keras Format
# -----------------------------
hybrid_model.save("trained_hybrid_model.keras")
print("Model saved as 'trained_hybrid_model.keras'")


Combined DataFrame shape: (20801, 40)
Optimized Filtering Parameters:
 Process Variance: 0.0006715429033260829
 Measurement Variance: 0.05235353908705457
 SG Window Length: 51
 SG Polyorder: 1
Filtering parameters saved to filtering_params.json
Processed data shape: (19718, 54)
Extracted features shape: (787, 17)
Sequence shape: (778, 10, 13) Labels shape: (787, 17)
Training Hybrid CNN-RNN Model...
Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.5740 - loss: 0.6884 - precision: 0.1807 - recall: 0.4988 - val_accuracy: 0.5641 - val_loss: 0.6833 - val_precision: 0.2872 - val_recall: 0.9643
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6101 - loss: 0.6065 - precision: 0.2811 - recall: 0.7443 - val_accuracy: 0.5897 - val_loss: 0.6667 - val_precision: 0.2955 - val_recall: 0.9286
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6981 - loss: 0.5392 - precision: 0.4033 - recall: 0.8408 - val_accuracy: 0.6667 - val_loss: 0.5839 - val_precision: 0